In [ ]:
import zipfile
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Step 1: Specify the path to the zip file and the extraction directory
zip_file_path = 'train.zip'  # Specify the path to your 'train.zip' file
extraction_path = 'dataset/train/'  # Specify the directory where you want to extract the dataset

# Create the extraction directory if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

# Step 2: Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# After running this code, you should have all images directly in the 'dataset/train' directory.

# Step 3: Load and Preprocess the Data
def preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = preprocess_input(img)
    return img

X = []
y = []

train_dir = 'dataset/train'

for img_file in os.listdir(train_dir):
    img_path = os.path.join(train_dir, img_file)
    img = preprocess_image(img_path)
    X.append(img)

    # Extract the label from the image filename ('cat' or 'dog')
    label = img_file.split('.')[0]
    if label == 'cat':
        y.append(0)  # 0 represents 'cat'
    elif label == 'dog':
        y.append(1)  # 1 represents 'dog'

X = np.array(X)
y = np.array(y)

# Step 4: Flatten and Split the Data
X_flatten = X.reshape(X.shape[0], -1)
X_train, X_test, y_train, y_test = train_test_split(X_flatten, y, test_size=0.2, random_state=42)

# Step 5: Extract Features Using a Pre-trained CNN (VGG16)
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

X_train_features = vgg16.predict(X_train.reshape(-1, 224, 224, 3))
X_test_features = vgg16.predict(X_test.reshape(-1, 224, 224, 3))

# Step 6: Train the Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_features, y_train)

# Step 7: Make Predictions
y_pred = svm_classifier.predict(X_test_features)

# Step 8: Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(report)


In [ ]:
import cv2
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load and preprocess the image you want to classify
image_path = ''  # Specify the path to your image
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224))
img = preprocess_input(img)

# Extract features using VGG16
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
image_features = vgg16.predict(np.expand_dims(img, axis=0))

# Use the trained SVM classifier to make a prediction
prediction = svm_classifier.predict(image_features)

# Map the prediction to 'cat' or 'dog'
if prediction == 0:
    result = 'cat'
else:
    result = 'dog'

print(f'The image is a {result}.')
